In [35]:
import os
import pandas as pd
import numpy as np

os.makedirs('example', exist_ok=True)

Create a metadata table summarizing the GWAS datasets. Here's an example:

In [36]:
gwas_input_table = pd.read_csv("example_data/example_gwas_table.tsv", sep="\t")

gwas_input_table

,GWAS,leads,GWAS_path,chr_col,pos_col,beta_col,se_col,file_type,ref_col,alt_col
0,L_HDL_C_pct,example_data/test_lead.tsv,example_data/gwas_test.tsv,CHROM,GENPOS,Effect,StdErr,tsv,ALLELE0,ALLELE1


Define file paths for saving GWAS signals and summary files.

In [37]:
gwas_signals_directory = "example/gwas_signals"
gwas_summary_file = "example/gwas_summary.tsv"

Generate summary files for each signal and save individual signal data as `.pickle` files.

In [38]:

def approx_bf_estimates(variant, z, V, type, suffix=None, sdY=1, effect_priors={'quant': 0.15, 'cc': 0.2}):
    sd_prior = effect_priors['quant'] * sdY if type == "quant" else effect_priors['cc']
    
    r = sd_prior**2 / (sd_prior**2 + V)
    lABF = 0.5 * (np.log(1 - r) + (r * z**2))
    
    ret = pd.DataFrame({"variant": variant, 'lbf': lABF})
    
    if suffix is not None:
        ret.columns = [f"{col}.{suffix}" for col in ret.columns]

    ret.loc[:, 'lbf'] = pd.to_numeric(ret['lbf'], errors='coerce')

    ret_mat = pd.DataFrame(ret.set_index('variant')['lbf']).T

    return ret_mat

os.makedirs(gwas_signals_directory, exist_ok=True)

for gwas in gwas_input_table.itertuples(index=False):

    lead_variants = pd.read_csv(gwas.leads, sep="\t")

    gwas_name = gwas.GWAS

    summary_rows = []

    if gwas.file_type == "parquet":
        gwas_df = pd.read_parquet(
            gwas.GWAS_path
        )
    elif gwas.file_type == "tsv":
        gwas_df = pd.read_csv(
            gwas.GWAS_path, sep="\t"
        )
    else:
        print(f"Invalid file type for {gwas.GWAS_path}")
        continue

    for region in lead_variants.itertuples(index=False):
        chrom = getattr(region, gwas.chr_col)
        region_start = getattr(region, gwas.pos_col) - 500_000
        region_end = getattr(region, gwas.pos_col) + 500_000

        region_gwas = gwas_df[gwas_df[gwas.chr_col] == chrom]

        chrom_label = "X" if chrom == 23 else str(chrom)

        region_signal = region_gwas[(region_gwas[gwas.pos_col] >= region_start) & (region_gwas[gwas.pos_col] <= region_end)]

        region_signal["variant"] = (
            "chr" + chrom_label + "_" +
            region_signal[gwas.pos_col].astype(int).astype(str) + "_" +
            region_signal[gwas.ref_col].astype(str) + "_" +
            region_signal[gwas.alt_col].astype(str)
        )

        region_signal["z"] = region_signal[gwas.beta_col] / region_signal[gwas.se_col]
        region_signal["V"] = region_signal[gwas.se_col] ** 2

        mat = approx_bf_estimates(region_signal["variant"],region_signal["z"], region_signal["V"], "quant") 
        
        signal_strength = mat.T["lbf"].max()
        variant_id = mat.T["lbf"].idxmax()

        if signal_strength < 5:
            continue

        signal = f"{gwas_name}_chr{chrom_label}:{region_start}-{region_end}"

        summary_data = pd.DataFrame([{
            'signal': signal,
            'chromosome': chrom_label,
            'location_min': region_start,
            'location_max': region_end,
            'signal_strength': signal_strength,
            'lead_variant': variant_id
        }])

        header_needed = not os.path.exists(gwas_summary_file)  
        summary_data.to_csv(gwas_summary_file, sep='\t', mode='a', header=header_needed, index=False)

        output_file_name = f"{signal}.pickle"
        output_file_path = os.path.join(gwas_signals_directory, output_file_name)

        mat.to_pickle(output_file_path)

/var/folders/g4/mx686c290ms3_t_t0prfdknc0000gn/T/ipykernel_1328/825445506.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  region_signal["variant"] = (
/var/folders/g4/mx686c290ms3_t_t0prfdknc0000gn/T/ipykernel_1328/825445506.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  region_signal["z"] = region_signal[gwas.beta_col] / region_signal[gwas.se_col]
/var/folders/g4/mx686c290ms3_t_t0prfdknc0000gn/T/ipykernel_1328/825445506.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

The summary file will have the following structure:

In [39]:
pd.read_csv(gwas_summary_file, sep='\t')

,signal,chromosome,location_min,location_max,signal_strength,lead_variant
0,L_HDL_C_pct_chrX:153853564-154853564,X,153853564,154853564,36.674323,chrX_154353564_A_G


Each signal `.pickle` file will contain log Bayes factors (lbf) as shown below:

In [40]:
pd.read_pickle("example/gwas_signals/L_HDL_C_pct_chrX:153853564-154853564.pickle")

variant,chrX_153853745_C_A,chrX_153853745_C_T,chrX_153855200_G_A,chrX_153855237_A_C,chrX_153855261_C_T,chrX_153855525_G_A,chrX_153855834_G_A,chrX_153855847_C_T,chrX_153856109_C_T,chrX_153856795_T_A,...,chrX_154852253_G_A,chrX_154852266_C_T,chrX_154852488_C_T,chrX_154852865_C_T,chrX_154852868_C_T,chrX_154852903_A_C,chrX_154853089_C_A,chrX_154853104_T_C,chrX_154853179_G_A,chrX_154853402_G_A
lbf,-0.89635,-2.834458,-0.059966,0.327294,0.228286,-0.833013,-2.840194,-0.98884,-2.572531,-1.707554,...,-1.029522,-0.370217,-0.772423,0.133273,-0.901668,-0.401849,-0.425781,-0.356414,-0.262577,-0.369968


Repeat the same steps for the second dataset (e.g., eQTL).

In [41]:
def process_gwas_file(file_path, directory_name, mat_dir, summary_file_path):
    os.makedirs(mat_dir, exist_ok=True)

    try:
        df = pd.read_csv(file_path, sep='\t', low_memory=False, on_bad_lines='skip')
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        return

    trait_data = df.groupby(df['molecular_trait_id'])
    
    for trait_id, group in trait_data:
        for i in range(1, 11):
            lbf_column = f'lbf_variable{i}'
            if lbf_column in group.columns:
                process_signal(group, directory_name, trait_id, mat_dir, lbf_column, i, summary_file_path)

def process_signal(group, directory_name, trait_id, mat_dir, lbf_column, lbf_index, summary_file_path):
    df_filtered = group[['molecular_trait_id', 'region', 'variant', 'chromosome', 'position', lbf_column]].copy()
    df_filtered.rename(columns={lbf_column: 'lbf'}, inplace=True)
    
    if (df_filtered['lbf'] == 0).all():
        return
    
    signal_strength = df_filtered['lbf'].abs().max()

    if signal_strength < 5:
        return
        
    chromosome = df_filtered['chromosome'].astype(str).iloc[0]

    location_min = df_filtered['position'].min()
    location_max = df_filtered['position'].max()

    signal = f"{directory_name}_{trait_id}_L{lbf_index}"
    output_file_name = f"{signal}.pickle"
    output_file_path = os.path.join(mat_dir, output_file_name)

    df_filtered['lbf'] = pd.to_numeric(df_filtered['lbf'], errors='coerce')

    mat1_df = pd.DataFrame(df_filtered.set_index('variant')['lbf']).T
    variant_id = mat1_df.T["lbf"].idxmax()

    mat1_df.to_pickle(output_file_path)

    summary_data = pd.DataFrame([{
        'signal': signal,
        'chromosome': chromosome,
        'location_min': location_min,
        'location_max': location_max,
        'signal_strength': signal_strength,
        'lead_variant': variant_id
    }])
    
    header_needed = not os.path.exists(summary_file_path)  
    summary_data.to_csv(summary_file_path, sep='\t', mode='a', header=header_needed, index=False)

eqtl_table = pd.read_csv("example_data/example_eQTL_table.tsv", sep='\t')
eqtl_signals_directory = "example/eqtl_signals"
os.makedirs(eqtl_signals_directory, exist_ok=True)

eqtl_summary_file = "example/eqtl_summary.tsv"

for row in eqtl_table.itertuples(index=False):
    dataset_id = row.id
    file_path = row.path
    process_gwas_file(file_path, dataset_id, eqtl_signals_directory, eqtl_summary_file)

Format the saved signals using the `gpu-coloc` formatting tool:

In [42]:
!gpu-coloc -f --input example/gwas_signals --output example/formatted_gwas --input_summary example/gwas_summary.tsv --output_summary example/gwas_files_summary.tsv
!gpu-coloc -f --input example/eqtl_signals --output example/formatted_eqtls --input_summary example/eqtl_summary.tsv --output_summary example/eqtl_files_summary.tsv

Processing chromosomes: 100%|█████████████████████| 1/1 [00:00<00:00,  1.19it/s]
Done.
Processing chromosomes: 100%|█████████████████████| 1/1 [00:00<00:00,  3.48it/s]
Done.


Run `gpu-coloc` using the following command:

In [43]:
!gpu-coloc -r --dir1 example/formatted_eqtls --dir2 example/formatted_gwas --results example/example_results.tsv --p12 1e-6 --H4 0.8

processing met:   0%|                                     | 0/2 [00:00<?, ?it/s]

running files:   0%|                                      | 0/1 [00:00<?, ?it/s]


All chunk pairs:   0%|                                    | 0/1 [00:00<?, ?it/s]


All chunk pairs: 100%|████████████████████████████| 1/1 [00:00<00:00,  6.25it/s]


                                                                                

running files: 100%|██████████████████████████████| 1/1 [00:00<00:00,  5.84it/s]

                                                                                
processing met:  50%|██████████████▌              | 1/2 [00:00<00:00,  5.80it/s]

running files:   0%|                                      | 0/1 [00:00<?, ?it/s]Possible error in trim function


                                                                                
chromosomes: 100%|████████████████████████████████| 1/1 [00:00<00:00,  1.13it/s]
chromosomes: 0it [00:00, ?it/s]


The results can be viewed in the final output file:

In [44]:
pd.read_csv("example/example_results.tsv", sep="\t")

,PP.H3,PP.H4,signal1,lead1,signal2,lead2
0,0.102638,0.897362,QTD000141_ENSG00000147403_L1,chrX_154356517_C_A,L_HDL_C_pct_chrX:153853564-154853564,chrX_154353564_A_G
